In [1]:
import requests
from datetime import datetime
import pandas as pd

class CommentsReply:
    def get_comments(self,owner_id, post_id, access_token):
        params = {
            'owner_id': owner_id,
            'post_id': post_id,
            'access_token': access_token,
            'v': '5.131',
            'extended': 1,
            'count': 100,
            'need_likes': 1
        }

        comments = []

        response = requests.get('https://api.vk.com/method/wall.getComments', params=params)
        data = response.json()

        if 'response' in data:
            for item in data['response']['items']:
                item['date'] = datetime.utcfromtimestamp(item['date']).strftime('%Y-%m-%d %H:%M:%S')
                if 'likes' in item:
                    item['likes_count'] = item['likes']['count']
                comments.append(item)
                if item['thread']['count'] > 0:
                    params['comment_id'] = item['id']
                    subcomments = self.get_subcomments(owner_id, post_id, access_token, params)
                    comments.extend(subcomments)

        return comments

    def get_subcomments(self, owner_id, post_id, access_token, params):
        subcomments = []

        response = requests.get('https://api.vk.com/method/wall.getComments', params=params)
        data = response.json()

        if 'response' in data:
            for item in data['response']['items']:
                item['date'] = datetime.utcfromtimestamp(item['date']).strftime('%Y-%m-%d %H:%M:%S')
                if 'likes' in item:
                    item['likes_count'] = item['likes']['count']
                subcomments.append(item)

        return subcomments

    def comments_to_dataframe(self, comments):
        df = pd.DataFrame(comments)
        df = df[['id', 'date', 'text', 'post_id', 'parents_stack', 'likes_count']]
        return df

In [2]:

comments_parser = CommentsReply()

owner_id='-129354225',
post_id='332329'
token='...'

comments = comments_parser.get_comments(owner_id, post_id, token)

df = comments_parser.comments_to_dataframe(comments)
pd.options.display.max_colwidth = 15
print(df.head(5))

       id            date            text  post_id parents_stack  likes_count
0  332330  2024-03-13 ...  Какая красо...   332329            []           47
1  332371  2024-03-14 ...  [id5588054|...   332329      [332330]            2
2  332331  2024-03-13 ...  Всю зиму жд...   332329            []           26
3  332332  2024-03-13 ...  Очень краси...   332329            []           12
4  332333  2024-03-13 ...  Великолепно...   332329            []           22
